In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch

this procedure is really like minecraft

In [3]:
# 6 direction of cube
directions = torch.tensor(
    [
        [+1, 0, 0],
        [-1, 0, 0],
        [0, +1, 0],
        [0, -1, 0],
        [0, 0, +1],
        [0, 0, -1],
    ]
)

In [31]:
directions.tolist()

[[1, 0, 0], [-1, 0, 0], [0, 1, 0], [0, -1, 0], [0, 0, 1], [0, 0, -1]]

In [4]:
i = torch.tensor(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
)
y90 = torch.tensor(
    [
        [0, 0, +1],
        [0, +1, 0],
        [-1, 0, 0],
    ]
)
z90 = torch.tensor(
    [
        [0, -1, 0],
        [1, 0, 0],
        [0, 0, +1],
    ]
)

In [5]:
rot = torch.stack(
    [
        i,
        y90 @ y90,
        z90,
        z90 @ z90 @ z90,
        y90 @ y90 @ y90,
        y90,
    ],
    dim=0,
)
rot.shape

torch.Size([6, 3, 3])

In [6]:
torch.einsum("ijk,kl->ijl", rot, directions[0].reshape(-1, 1)).reshape(-1, 3)

tensor([[ 1,  0,  0],
        [-1,  0,  0],
        [ 0,  1,  0],
        [ 0, -1,  0],
        [ 0,  0,  1],
        [ 0,  0, -1]])

In [7]:
# the position of verts of a cube of size 2 in the origin, for the face of 1, 0, 0
xverts = torch.tensor(
    [
        [1, 1, 1],
        [1, -1, 1],
        [1, -1, -1],
        [1, 1, -1],
    ]
)

xfaces = torch.tensor(
    [
        [0, 1, 3],
        [1, 2, 3],
    ]
)

In [8]:
faces = torch.arange(6).reshape(-1, 1, 1) * 4 + xfaces.reshape(1, 2, 3)  # direction, face, face_idx
faces.shape, faces.max() + 1

(torch.Size([6, 2, 3]), tensor(24))

In [9]:
verts = torch.einsum('nij,vj->nvi', rot, xverts) # direction, number of verts, coord, 6x4x3

In [10]:
from core.integrate.utils.voxel_ops import discrete2hash, hash2discrete

In [11]:
verts.shape

torch.Size([6, 4, 3])

In [39]:
((new_verts + 1) // 2).tolist()

[[0, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 1, 0],
 [1, 1, 1]]

In [12]:
unique_hash, inverse = discrete2hash(verts).unique(sorted=True, return_inverse=True)

In [13]:
new_verts = hash2discrete(unique_hash)

In [14]:
new_face = inverse.reshape(-1)[faces]

In [43]:
new_face.tolist()

[[[7, 5, 6], [5, 4, 6]],
 [[2, 0, 3], [0, 1, 3]],
 [[3, 7, 2], [7, 6, 2]],
 [[5, 1, 4], [1, 0, 4]],
 [[3, 1, 7], [1, 5, 7]],
 [[6, 4, 2], [4, 0, 2]]]

In [15]:
import open3d as o3d

draw = o3d.visualization.EV.draw

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [30]:
selected = torch.tensor(
    [
        1,
        0,
        0,
        0,
        0,
        0,
    ],
    dtype=bool,
)
selected = torch.ones_like(selected)
selected_verts_idx, selected_faces = new_face[selected].unique(sorted=True, return_inverse=True)
selected_verts = new_verts[selected_verts_idx]

mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(selected_verts.double().numpy()),
    triangles=o3d.utility.Vector3iVector(selected_faces.reshape(-1, 3).numpy()),
)
mesh.compute_vertex_normals()
draw([mesh])

In [27]:
selected_faces

tensor([[[3, 1, 2],
         [1, 0, 2]]])

In [28]:
new_face.shape

torch.Size([6, 2, 3])